# import_data

In [1]:
import pandas as pd
import joblib
X_train = pd.read_csv("/ml_data/X_train.csv").drop(columns = "Unnamed: 0")
y_train = pd.read_csv("/ml_data/y_train.csv").drop(columns = "Unnamed: 0")
X_test = pd.read_csv("/ml_data/X_test.csv").drop(columns = "Unnamed: 0")
y_test = pd.read_csv("/ml_data/y_test.csv").drop(columns = "Unnamed: 0")
model = joblib.load("/artifact/mlruns/941649382202349625/e311be07056d4a4fa7413ec46830e13b/artifacts/model/model.pkl")

if str(model[1]) ==  'KNeighborsClassifier()':
    pred_model = model[-1].predict_proba
else:
    pred_model = model[-1].predict

In [2]:
str(model[1]) 

'KNeighborsClassifier()'

# local explainer

In [3]:
from tqdm import tqdm_notebook

In [1]:
def _explainer(task, target):
    if task == "classification":
        class_names = y_train[target].unique()
        feature_names = list(X_train.columns)

        # Fit the Explainer on the training data set using the LimeTabularExplainer 
        explainer = LimeTabularExplainer(X_train.values, feature_names = feature_names, class_names = class_names, mode = task)
    return explainer

In [17]:
import numpy as np
import warnings
from lime import submodular_pick
from tqdm import tqdm_notebook
from lime.lime_tabular import LimeTabularExplainer
import pandas as pd
import joblib

X_train = pd.read_csv("/ml_data/X_train.csv").drop(columns = "Unnamed: 0")
y_train = pd.read_csv("/ml_data/y_train.csv").drop(columns = "Unnamed: 0")
X_test = pd.read_csv("/ml_data/X_test.csv").drop(columns = "Unnamed: 0")
y_test = pd.read_csv("/ml_data/y_test.csv").drop(columns = "Unnamed: 0")
feature_names = list(X_train.columns)
model = joblib.load("/artifact/mlruns/941649382202349625/e311be07056d4a4fa7413ec46830e13b/artifacts/model/model.pkl")
task = "classification"
target = "quality"

explainer = _explainer(task, target)

if str(model[1]) ==  'KNeighborsClassifier()':
    pred_model = model[-1].predict_proba
else:
    pred_model = model[-1].predict
    
def check_file():
    path = Path("/model_interpret")
    return path.is_file()
    
# def _explainer(task, target):
#     if task == "classification":
#         class_names = y_train[target].unique()
#         feature_names = list(X_train.columns)

#         # Fit the Explainer on the training data set using the LimeTabularExplainer 
#         explainer = LimeTabularExplainer(X_train.values, feature_names = feature_names, class_names = class_names, mode = task)
#     return explainer

def return_x(int_x):
    return explaination.domain_mapper.exp_feature_names[int_x]

def return_x_2(int_x):
    return sp_obj.explanations[i].domain_mapper.exp_feature_names[int_x]

def _to_csv_local(explanation):
    all_df = []
    cols = []
    values = []
    for x,y in list(explaination.local_exp.values())[0]:
        cols.append(return_x(x))
        values.append(y)
    arr = np.array(values).reshape((1, len(list(explaination.local_exp.values())[0])))
    return pd.DataFrame(arr , columns = cols)

def _to_csv_global(explanation):
    sp_obj = explanation
    all_df = []
    for i in range(0,5):
        sp_obj.explanations[i].domain_mapper.exp_feature_names
        cols = []
        values = []
        for x,y in list(sp_obj.explanations[i].local_exp.values())[0]:
            cols.append(return_x(x))
            values.append(y)
        arr = np.array(values).reshape((1, len(list(sp_obj.explanations[i].local_exp.values())[0])))
        all_df.append(pd.DataFrame(arr , columns = cols))
    return pd.concat(all_df, ignore_index =True)

def save_to_csv(task, target):
    all_df_local = []
    for i in tqdm_notebook(range(len(X_test))):
        explaination = explainer.explain_instance(X_test.iloc[i], pred_model)
        all_df_local.append(_to_csv_local(explaination))

    sp_obj = submodular_pick.SubmodularPick(explainer, X_train.values, pred_model, sample_size= 500, num_features= len(feature_names), num_exps_desired = 5)
    all_df_global = _to_csv_global(sp_obj)
    pd.concat(all_df_local, ignore_index = True).fillna(0).to_csv("/model_interpret/local.csv")
    all_df_global.fillna(0).to_csv("/model_interpret/global.csv")


In [21]:
explaination = explainer.explain_instance(X_test.iloc[2], pred_model)

_to_csv_local(explaination)

,fixed acidity,chlorides,total sulfur dioxide,residual sugar,sulphates,free sulfur dioxide,alcohol,volatile acidity,pH,citric acid
0,0.000378,0.000239,-0.000215,-0.000212,-0.000207,-0.000207,-0.000206,0.000037,-0.000035,0.000018


In [23]:
jaa = explainer.explain_instance(X_test.iloc[2], pred_model).__dict__

In [29]:
jaa

{'random_state': None,
 'mode': 'classification',
 'domain_mapper': <lime.lime_tabular.TableDomainMapper at 0x7f305c95d850>,
 'local_exp': {1: [(6, -0.0005295314461779517),
   (4, 0.0005221340236278763),
   (3, -0.0005132399733361935),
   (5, -0.0005020576927817447),
   (7, -0.0003412891849203574),
   (9, -0.00030490712222912736),
   (10, -0.0003015177870110326),
   (0, 0.00027561339943824064),
   (8, 0.00020505698872843402),
   (2, -0.00011090011420983774)]},
 'intercept': {1: 0.0007951500250012664},
 'score': 0.003877039840493035,
 'local_pred': array([-0.00080549]),
 'class_names': [5.0, 7.0, 6.0, 8.0, 3.0, 4.0],
 'top_labels': None,
 'predict_proba': array([0. , 0. , 0.4, 0.4, 0.2, 0. ])}

In [35]:
jaa["domain_mapper"].__dict__

{'exp_feature_names': ['fixed acidity',
  'volatile acidity',
  'citric acid',
  'residual sugar',
  'chlorides',
  'free sulfur dioxide',
  'total sulfur dioxide',
  'density',
  'pH',
  'sulphates',
  'alcohol'],
 'discretized_feature_names': ['7.90 < fixed acidity <= 9.30',
  '0.39 < volatile acidity <= 0.52',
  'citric acid > 0.42',
  '1.90 < residual sugar <= 2.20',
  'chlorides > 0.09',
  '13.00 < free sulfur dioxide <= 21.00',
  'total sulfur dioxide > 60.00',
  'density > 1.00',
  'pH <= 3.21',
  'sulphates > 0.72',
  'alcohol <= 9.50'],
 'feature_names': ['fixed acidity',
  'volatile acidity',
  'citric acid',
  'residual sugar',
  'chlorides',
  'free sulfur dioxide',
  'total sulfur dioxide',
  'density',
  'pH',
  'sulphates',
  'alcohol'],
 'feature_values': ['8.60',
  '0.49',
  '0.51',
  '2.00',
  '0.42',
  '16.00',
  '62.00',
  '1.00',
  '3.03',
  '1.17',
  '9.00'],
 'feature_indexes': None,
 'scaled_row': array([1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.]),
 'all_catego

In [20]:
save_to_csv("classification", "quality")

/tmp/ipykernel_275/1374112070.py:70: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  for i in tqdm_notebook(range(len(X_test))):


  0%|          | 0/252 [00:00<?, ?it/s]

In [7]:
explainer = _explainer("classification", "quality")
explaination = explainer.explain_instance(X_test.iloc[2], pred_model)
_to_csv_local(explaination)

,fixed acidity,free sulfur dioxide,citric acid,residual sugar,total sulfur dioxide,density,alcohol,chlorides,pH,volatile acidity
0,-0.000294,-0.00029,-0.000284,-0.000279,-0.000276,-0.000118,0.000094,-0.000092,0.000089,-0.000087
